# Datan analysointi K2020_2 - Tehtävät kerta 3

Palauta Optiman palautuslaatikkoon viimeistään su 16.2. Kirjoita palautuslaatikon kommenttikenttään, mitä tehtävät olet tehnyt kokonaan/osittain.

**Tee tehtävät ilman for-silmukoita tms.**

**Opiskelijan nimi:**


## 1. Maakunnat





Lue DataFrameen tiedosto jossa on tilastotietoja Suomen kunnnista (vuosilta 2017-19). Kenttäerotin puolipiste ja deimaalierotin pilkku.

https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kunnat.txt


a) Laske kuinka paljon maapinta-alaa on kussakin seutukunnassa ja tulosta maapinta-alaltaan 5 suurinta seutukuntaa suuruusjärjestyksessä.

b) Laske kuinka monta prosenttia kunkin maakunnan väestöstä asuu kaupungeissa (kuntamuoto) ja tulosta 5 suurinta %-osuutta. 
 
c) Laske ruotsinkielisten osuus maakunnittain (huom. et voi laskea prosenttiluvuista keskiarvoa, koska kuntien väkiluvut vaihtelevat). Tulosta 5 suurinta osuutta.




In [1]:
import pandas as pd
df = pd.read_csv('https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kunnat.txt', sep=';', decimal=',')

print('\n---- a) Suurimmat seutukunnat  -----------')
print(df['maapinta-ala'].groupby(df['seutukunta']).sum().sort_values(ascending=False).head(5))

print('\n---- b) % osuus kaupungeissa  -----------')
maakuntaväkiluku = (df['Väkiluku'].groupby(df['maakunta']).sum())
kaupunki = df.loc[df['kuntamuoto']=='Kaupunki',['kunta','maakunta','Väkiluku']]
print(((kaupunki['Väkiluku'].groupby(kaupunki['maakunta']).sum())/maakuntaväkiluku*100).sort_values(ascending=False).head(5))

print('\n---- c) ruotsinkielisten osuus  -----------')
df['ruotsinkielisiä']=df['Ruotsinkielisten osuus%']*df['Väkiluku']/100
ruotsi=(df['ruotsinkielisiä'].groupby(df['maakunta']).sum())
print((ruotsi/maakuntaväkiluku*100).sort_values(ascending=False).head(5))




---- a) Suurimmat seutukunnat  -----------
seutukunta
Pohjois-Lapin seutukunta    31893.91
Itä-Lapin seutukunta        20548.92
Tunturi-Lapin seutukunta    20512.50
Kehys-Kainuun seutukunta    13959.50
Rovaniemen seutukunta       11035.13
Name: maapinta-ala, dtype: float64

---- b) % osuus kaupungeissa  -----------
maakunta
Kymenlaakso        90.171177
Uusimaa            86.882115
Pirkanmaa          84.496840
Varsinais-Suomi    82.700352
Etelä-Pohjanmaa    81.707298
Name: Väkiluku, dtype: float64

---- c) ruotsinkielisten osuus  -----------
maakunta
Ahvenanmaa         86.770785
Pohjanmaa          49.459568
Keski-Pohjanmaa     9.040979
Uusimaa             7.840641
Varsinais-Suomi     5.655749
dtype: float64


## 2.  access.log

Lue erään palvelimen (3 tunnin ajan) access.log-tiedosto (15 Mt zip, noin 600 000 riviä) DataFrameen  
http://gpspekka.kapsi.fi/accesslog2.zip  
2-kerran teoriamateriaalissa on ohjeita datan lukemiseen.

Kentät  
* ip-osoite
* ident-id (-)
* userid (-)
* time
* request
* status
* size
* referer
* user agent


Parsi aikakentästä tunti ja minuutit ja laske jokaiselle minuutille kuinka monta eri ip-osoitetta on kyseisenä päivänä sivulla vieraillut.   

Tulosta listaa suuruusjärjestyksessä eli eniten eri ip-osoitteita sisältänyt minuutti ensin. 

Alkua:
```
12:55    792
12:51    789
12:54    789
12:53    787
12:56    786
12:52    783
14:00    773
14:01    769
13:59    762
12:50    760

```


In [2]:
import pandas as pd

def delete (x):
    return x[13:-3]

df = pd.read_csv('http://gpspekka.kapsi.fi/accesslog2.zip',
                sep=' ',
                usecols=[0,3],
                quotechar='"',
                names=['ip-osoite', 'ident-id (-)', 'userid (-)', 'time', 'timezone', 'request', 'status', 'size', 'referer', 'user agent'],
                converters={'time' : delete}
                )
df = df['ip-osoite'].groupby(df['time']).nunique().sort_values(ascending=False)
df

time
12:55    792
12:51    789
12:54    789
12:53    787
12:56    786
        ... 
12:03    273
12:04    268
12:02    267
12:01    251
12:00    246
Name: ip-osoite, Length: 180, dtype: int64

## 3. Kone

Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.csv  (11 Mt) on erään koneen toiminnasta dataa 10 sekunnin välein. Aikaleiman lisäksi datassa on sarake `katko`, joka on 1 kun koneen toiminta on ollut katkoksissa ja 0 kun kone on ollut toiminnassa.

Poimi datasta tietoa katkoista eli **peräkkäisistä 1-riveistä**. Listaa kustakin katkosta:  
  * monesko katko on kyseessä
  * koska katko alkoi (eli ensimmäinen aikaleima)
  * koska katko loppui (eli viimeinen aikaleima)
  * kauanko katko kesti tunteina (rivien määrä jaettuna 360:lla).
  
10 ensimmäistä katkoa pitäisi näyttää tältä:
![katkot](https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.png)  

In [65]:
import pandas as pd
df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/kone.csv')
df['katkosumma']=df['katko'].cumsum()
df['katko_alkaa']=(df['katko']>0)&(df['katko'].shift(1)==0)
df['alku']=df.loc[df['katko_alkaa']==True,'aika']
df['katkonro']=df['katko_alkaa'].cumsum()


df2=pd.DataFrame(df[df['katko']>0].groupby('katkonro')['katko'].count()/360)
df2['alku_aika']=(df[df['katko']>0].groupby('katkonro')['aika'].first())
df2['loppu_aika']=(df[df['katko']>0].groupby('katkonro')['aika'].last())

df2



,katko,alku_aika,loppu_aika
katkonro,,,
1,0.136111,2019-03-18T11:26:30.000000Z,2019-03-18T11:34:30.000000Z
2,0.127778,2019-03-18T11:35:30.000000Z,2019-03-18T11:43:00.000000Z
3,0.191667,2019-03-18T14:05:30.000000Z,2019-03-18T14:16:50.000000Z
4,0.119444,2019-03-18T21:49:00.000000Z,2019-03-18T21:56:00.000000Z
5,0.086111,2019-03-18T22:51:20.000000Z,2019-03-18T22:56:20.000000Z
...,...,...,...
96,0.105556,2019-04-27T10:16:40.000000Z,2019-04-27T10:22:50.000000Z
97,0.130556,2019-04-27T10:41:30.000000Z,2019-04-27T10:49:10.000000Z
98,0.036111,2019-04-27T14:22:40.000000Z,2019-04-27T14:24:40.000000Z


## 4. Lumeton alkutalvi?

Lataa DataFrameen csv-tiedosto  
http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/saajkl20200118.csv

Tiedostossa on päivittäisiä säähavaintoja Jyväskylän lentoasemalta vuodesta 1959.

Laske tietojen perusteella jokaiselle "alkutalvelle" 1. marraskuuta - 18. tammikuuta  **"lumisumma"** jossa on alkutalven jokaisen vuorokauden lumensyvyys-lukema laskettu yhteen (ei ole mikään oikea meteorologinen termi).

Huomioita:
* lumisumma pitää laske talvittain, ei siis vuosittain. Tee uusi sarake joka kertoo mihin (alku)talveen kyseinen vuorokausi kuuluu. Siis tammikuu 2020 kuuluu samaan alkutalveen kuin joulukuu 2019.

* Ilmatieteen laitos käyttää arvoa -1 lumensyvyyden arvona jos "lunta ei ole näkyvissäkään". Ei haluta kuitenkaan miinustaa yhtä näinä päivinä, joten korvaa -1:t nollilla.

* Lisäksi joinain päivinä lumensyvyys todella (jostain syystä) puuttuu. Jos nämä jätetään laskusta pois, kyseisen talven lukema jää liian pieneksi (jos lunta on kuitenkin ollut). Lienee turvallista olettaa että lunta on varmaan ollut suunnilleen saman verran kuin edellisenä päivänä, joten täytä NaN-arvot edellsillä arvoilla

Anna tuloksena DataFrame, jossa on indeksinä talvi (joko muodossa 2019 tai 2019-2020) ja sarakkeina 
* alkutalventalven lumisumma 
* sijoitus lumisummien joukossa (lumisin alkutalvi = 1) 
* alkutalven lumipäivien lukumäärä (eli kuinka monena päivänä lumensyvyys on ollut suurempi kuin nolla)
* alkutalven suurin lumensyvyys.

mallia alusta ja lopusta:  
 ```
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
1959-1960         1292    20          61       38
1960-1961         1305    19          61       52
1961-1962         1116    28          54       41
1962-1963         1065    30          65       35
1963-1964          850    41          69       22
           Lumensyvyys  sija  lumipaivia  maksimi
Talvi                                            
2015-2016          139    61          31       14
2016-2017          737    50          69       18
2017-2018         1476    15          62       49
2018-2019          568    53          39       31
2019-2020          856    39          68       30
 ```


In [133]:
import pandas as pd
df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/saajkl20200118.csv',
                 sep=',',
                usecols=[0,1,2,6])
df['Lumensyvyys'].replace([-1],[0],inplace=True)
df['Lumensyvyys'].fillna(df['Lumensyvyys'].shift(-1),inplace=True)

#df['pvm'] = pd.to_datetime(df.Vuosi*10000+df.Kk*100+df.Pv,format='%Y%m%d')
df['alkutalvi']=((df['Kk']==1)&((df['Pv']>=1)&(df['Pv']<=18)))|(df['Kk'].isin([11,12]))
df['talvialku']=(df['Kk']==11)&(df['Pv']==1)
df['talvinro']=df['talvialku'].cumsum()+1958
df.loc[df['alkutalvi']==True,'Lumi']=df['Lumensyvyys']
Lumisumma = df['Lumi'].groupby(df['talvinro']).sum()
Lumipaivia = pd.DataFrame(df[df['Lumi']>0].groupby(df['talvinro']).count())

df2 = pd.DataFrame()
df2['Lumensyvyys']=df['Lumi'].groupby(df['talvinro']).sum()
df2['sija'] = Lumisumma.rank(method='first',ascending=False)
df2['Lumipäiviä'] = Lumipaivia['Lumi']
df2['max']=df['Lumi'].groupby(df['talvinro']).max()
df2

                                 

,Lumensyvyys,sija,Lumipäiviä,max
talvinro,,,,
1958,0.0,62.0,NaN,NaN
1959,1292.0,19.0,61.0,38.0
1960,1275.0,21.0,59.0,52.0
1961,1116.0,28.0,54.0,41.0
1962,1065.0,30.0,65.0,35.0
...,...,...,...,...
2015,139.0,61.0,31.0,14.0
2016,737.0,50.0,69.0,18.0
2017,1476.0,15.0,62.0,49.0


## 5. Liiga

Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/liiga.txt on SM-liigan (nyk. Liiga) runkosarjan ottelutukset 2010-luvulta.  

huom. -sarakkeessa on merkintä JA tai VL, jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa.

Voittanut (enemmän maaleja tehnyt joukkue) saa 3 pistettä ja hävinnyt 0 pistettä, paitsi jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa, voittanut joukkue saa 2 pistettä ja hävinnyt yhden pisteen.



Generoi tulosten perusteella "vuosikymmenen sarjataulukko", jossa on kullekin joukkueelle omana sarakkeenaan
* sijoitus
* joukkueen nimi
* pelatut ottelut
* 3 pisteen voitot
* 2 pisteen voitot (JA/VL)
* 1 pisteen tappiot (JA/VL)
* 0 pisteen tappiot
* tehdyt maalit ja päästetyt maalit (samassa sarakkeessa)
* pisteet



Lajittele sarjataulukko pisteiden mukaan (eniten pisteitä ekana). Tasapisteissä olevat joukkueet menevät maalieron (tehdyt maalit - päästetyt maalit) mukaiseen järjestykseen, ja jos sekin on sama niin tehdyt maalit ratkaisevat.

tavoiteltu lopputulos:

```
    sija   joukkue  ott   v3  v2  t1   t0     maalit  pist
0      1    Kärpät  574  295  70  51  158  1678-1244  1076
1      2       JYP  575  277  62  58  178  1614-1311  1013
2      3   Tappara  575  261  78  62  174  1598-1387  1001
3      4      HIFK  575  264  62  63  186  1637-1403   979
4      5     KalPa  574  224  69  68  213  1414-1400   878
5      6     Lukko  574  226  62  67  219  1516-1479   869
6      7     Ässät  574  215  64  56  239  1428-1501   829
7      8  Pelicans  574  215  57  56  246  1530-1595   815
8      9       HPK  575  198  80  60  237  1416-1498   814
9     10     SaiPa  574  201  66  71  236  1451-1576   806
10    11       TPS  572  193  59  78  242  1410-1596   775
11    12     Ilves  574  192  55  60  267  1426-1643   746
12    13     Blues  360  126  43  36  155    852-943   500
13    14   Jokerit  240  102  29  37   72    672-586   401
14    15     Sport  334   87  37  47  163   810-1028   382
15    16    KooKoo  275   79  32  42  122    664-809   343
16    17   Jukurit  215   56  21  34  104    486-603   244
```

(jos saat Ässille 832 pistettä, ohjelmakoodisi tulkitsee voittajan tuloksesta 11-3 väärin...) 


In [3]:
import pandas as pd
df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta3/liiga.txt',sep=',')
df['kotij']=df['ottelu'].str.split('-').str.get(0).str.strip()
df['vierasj']=df['ottelu'].str.split('-').str.get(1).str.strip()
df['kotim'] = df['tulos'].str.split('-').str.get(0).astype(int)
df['vierasm'] = df['tulos'].str.split('-').str.get(1).astype(int)
df['kotipist'] = 0
df['vieraspist'] = 0
df.loc[df['kotim']>df['vierasm'] , 'kotipist'] = 3                              # kotijoukkueen voitto: kotij = 3 pist
df.loc[(df['huom'].notnull()) & (df['kotim']>df['vierasm']), 'kotipist'] = 2    # kotijoukkueen voitto: jos JA/VL kotij = 2 piste
df.loc[(df['huom'].notnull()) & (df['kotim']<df['vierasm']), 'kotipist'] = 1    # kotijoukkueen tappio: jos JA/VL kotij = 1 piste

df.loc[df['kotim']<df['vierasm'] , 'vieraspist'] = 3                            # vierasjoukkueen voitto: vierasj = 3 pist
df.loc[(df['huom'].notnull()) & (df['kotim']<df['vierasm']), 'vieraspist'] = 2  # vierasjoukkueen voitto: jos JA/VL vierasj = 2 piste
df.loc[(df['huom'].notnull()) & (df['kotim']>df['vierasm']), 'vieraspist'] = 1  # vierasjoukkueen tappio: jos JA/VL vierasj = 1 piste

df2 = pd.DataFrame(df['kotipist'].groupby(df['kotij']).sum()+df['vieraspist'].groupby(df['vierasj']).sum())
#df2['sija'] = 0
df2['joukkue'] = (df['kotij'].unique())
df2['ott'] = pd.DataFrame(df['ottelu'].groupby(df['kotij']).count()+df['ottelu'].groupby(df['vierasj']).count())

df2['maalit'] = pd.DataFrame(df['kotipist'].groupby(df['kotij']).sum())
df2['pist'] = pd.DataFrame(df['kotipist'].groupby(df['kotij']).sum()+df['vieraspist'].groupby(df['vierasj']).sum())




#df2=pd.DataFrame(df[df['katko']>0].groupby('katkonro')['katko'].count()/360)
#print(pisteet.sort_values(ascending=False))

#print(df['kotipist'].groupby(df['kotij']).sum())
#print(df['vieraspist'].groupby(df['vierasj']).sum())
df2


                
 

,0,joukkue,ott,maalit,pist
kotij,,,,,
Blues,500,HIFK,360,321,500
HIFK,979,Ilves,575,551,979
HPK,814,KalPa,575,474,814
Ilves,746,Kärpät,574,410,746
JYP,1013,Lukko,575,579,1013
Jokerit,401,TPS,240,222,401
Jukurit,244,Blues,215,129,244
KalPa,878,HPK,574,492,878
KooKoo,343,Jokerit,275,197,343
